AWS versus WRF 15 km
======

Import custom modules for working with model and AWS data
----

In [1]:
from netCDF4 import Dataset
import pandas as pd
import numpy as np
from IPython.display import HTML

%load_ext autoreload
%autoreload 2
from model import Model
from GCNet import GCNet

Read WRF metadata
----

In [2]:
M = Model("wrf_geog.nc")
M.loadMeta()

minLon = -70 + 360
maxLon = -15 + 360
minLat = 58
maxLat = 86.5
M.setDomain( (minLat, maxLat, minLon, maxLon) )

2d dataset


Load AWS metadata
----

In [3]:
A = GCNet("site_info.nc")
A.loadMeta()
nSites = A.getNSites()

Find closest grid points to each AWS
-----

In [4]:
C = []
for awsIX in (range(nSites)):
    """ Basic AWS info """
    A.info(awsIX)
    awsName = A.getName( awsIX )
    awsLatLon = ( A.getLat( awsIX ), A.getLon( awsIX ))
    awsElev = A.getElev( awsIX )
    
    """ Find closest points """
    closest = M.closestPoints( awsLatLon )
    C.append( closest )
    
    """ Print info on closest points """
    print "Closest model points"
    P = []
    for d in closest:
        px = d['ixLon']
        py = d['ixLat']
        pdist = d['distance']
        plat  = M.getLat2d(py,px)
        plon  = M.getLon2d(py,px)
        pelev = M.getElev(py,px)
        perr = pelev - awsElev
        L = (py, px, pdist, plat, plon, pelev, perr)
        print "G(%d,%d): M: D = %.2f km, lat/lon/elev = %.2f/%.2f/%.2f m, elev_err = %.2f m" \
                     % L
        P.append( L ) 
    print "-"*80


Site 1: Swiss_Camp
Lat, Lon: 69.5732 310.7048
Elevation: 1176.0
Closest model points
G(93,74): M: D = 2.14 km, lat/lon/elev = 69.59/310.74/1195.45 m, elev_err = 19.45 m
G(93,73): M: D = 13.30 km, lat/lon/elev = 69.57/310.36/941.76 m, elev_err = -234.24 m
G(92,74): M: D = 13.65 km, lat/lon/elev = 69.46/310.80/1153.09 m, elev_err = -22.91 m
G(94,74): M: D = 16.33 km, lat/lon/elev = 69.72/310.68/1118.76 m, elev_err = -57.24 m
--------------------------------------------------------------------------------
Site 2: Crawford_Point1
Lat, Lon: 69.8819 313.0264
Elevation: 2022.0
Closest model points
G(94,80): M: D = 5.58 km, lat/lon/elev = 69.83/312.98/1969.98 m, elev_err = -52.02 m
G(95,80): M: D = 10.11 km, lat/lon/elev = 69.97/312.93/1991.25 m, elev_err = -30.75 m
G(94,81): M: D = 13.58 km, lat/lon/elev = 69.85/313.37/2045.52 m, elev_err = 23.52 m
G(95,81): M: D = 15.99 km, lat/lon/elev = 69.98/313.32/2070.36 m, elev_err = 48.36 m
-------------------------------------------------------------

Closest model points
G(62,99): M: D = 5.54 km, lat/lon/elev = 65.72/320.32/1097.90 m, elev_err = 219.90 m
G(63,99): M: D = 10.96 km, lat/lon/elev = 65.85/320.32/1456.87 m, elev_err = 578.87 m
G(62,100): M: D = 12.02 km, lat/lon/elev = 65.72/320.64/962.23 m, elev_err = 84.23 m
G(63,100): M: D = 15.30 km, lat/lon/elev = 65.85/320.65/1418.93 m, elev_err = 540.93 m
--------------------------------------------------------------------------------
Site 19: JAR3
Lat, Lon: 69.3954 309.6896
Elevation: 323.0
Closest model points
G(92,71): M: D = 1.04 km, lat/lon/elev = 69.39/309.68/365.32 m, elev_err = 42.32 m
G(93,71): M: D = 14.10 km, lat/lon/elev = 69.52/309.61/374.50 m, elev_err = 51.50 m
G(92,72): M: D = 14.26 km, lat/lon/elev = 69.41/310.05/601.05 m, elev_err = 278.05 m
G(92,70): M: D = 15.57 km, lat/lon/elev = 69.36/309.30/286.53 m, elev_err = -36.47 m
--------------------------------------------------------------------------------
Site 20: Aurora
Lat, Lon: 67.1352 312.7089
Elevation: 1798

Interpolate model elevation to AWS location
====

In [5]:
df = pd.DataFrame(columns=['Site #','Site name','Site lat','Site lon','Site elev','ElevCorr','ElevErr',
                           'Dry T Adj','Moist T Adj'],index=range(nSites))
for awsIX in (range(nSites)):
# for awsIX in ((0,1)):
    """ Basic AWS info """
    A.info(awsIX)
    awsName = A.getName( awsIX )
    awsLatLon = ( A.getLat( awsIX ), A.getLon( awsIX ))
    awsElev = A.getElev( awsIX )

    """ Perform the interpolation on elev from model metadata """
    closest = C[awsIX]
    Z = [ M.getElev( d[0], d[1] ) for d in zip( closest['ixLat'], closest['ixLon'] )]
    Znew = M.interpolate( Z, closest, awsLatLon )
    perr = Znew - awsElev
    print "\nNew elev = %0.2f, elev_err = %0.2f" % ( Znew, perr )
    dryAdAdj = perr/1000.*9.8
    mstAdAdj = perr/1000.*6.0
    row = ( awsIX+1, awsName, "%.2f" % awsLatLon[0], "%.2f" % awsLatLon[1], awsElev, \
            "%.2f" % Znew, "%.2f" % perr, \
            "%.2f" % dryAdAdj, "%.2f" % mstAdAdj )
    df.loc[awsIX] = row
    print "-"*80

Site 1: Swiss_Camp
Lat, Lon: 69.5732 310.7048
Elevation: 1176.0

New elev = 1163.27, elev_err = -12.73
--------------------------------------------------------------------------------
Site 2: Crawford_Point1
Lat, Lon: 69.8819 313.0264
Elevation: 2022.0

New elev = 1988.95, elev_err = -33.05
--------------------------------------------------------------------------------
Site 3: NASA-U
Lat, Lon: 73.8333 310.5047
Elevation: 2334.0

New elev = 2350.94, elev_err = 16.94
--------------------------------------------------------------------------------
Site 4: GITS
Lat, Lon: 77.1433 298.905
Elevation: 1869.0

New elev = 1885.53, elev_err = 16.53
--------------------------------------------------------------------------------
Site 5: Humboldt
Lat, Lon: 78.5266 303.1695
Elevation: 1995.0

New elev = 1975.90, elev_err = -19.10
--------------------------------------------------------------------------------
Site 6: Summit
Lat, Lon: 72.5794 321.4958
Elevation: 3199.0

New elev = 3242.02, elev_err 

In [6]:
df

,Site #,Site name,Site lat,Site lon,Site elev,ElevCorr,ElevErr,Dry T Adj,Moist T Adj
0,1,Swiss Camp,69.57,310.70,1176,1163.27,-12.73,-0.12,-0.08
1,2,Crawford Point1,69.88,313.03,2022,1988.95,-33.05,-0.32,-0.20
2,3,NASA-U,73.83,310.50,2334,2350.94,16.94,0.17,0.10
3,4,GITS,77.14,298.90,1869,1885.53,16.53,0.16,0.10
4,5,Humboldt,78.53,303.17,1995,1975.90,-19.10,-0.19,-0.11
5,6,Summit,72.58,321.50,3199,3242.02,43.02,0.42,0.26
6,7,Tunu-N,78.02,326.01,2052,2118.12,66.12,0.65,0.40
7,8,DYE-2,66.48,313.72,2099,2120.18,21.18,0.21,0.13
8,9,JAR1,69.50,310.32,932,873.04,-58.96,-0.58,-0.35
9,10,Saddle,66.00,315.50,2467,2485.59,18.59,0.18,0.11


In [7]:
# save the dataframe of AWS info
df.to_pickle('wrf_site_info.pkl' )

In [8]:
# print a DataFrame to HTML file (then print that to PDF in a browser!)
hout = HTML(df.to_html())
hfile = open("aws_table.htm","w")
hfile.write(hout.data)
hfile.close()